<a href="https://colab.research.google.com/github/Minwoo-study/AIPrograming/blob/main/melon_scraper_chart_%EB%AF%BC%EC%9A%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 멜론 차트 스크래핑

In [2]:
! pip install selenium Bs4 

     |████████████████████████████████| 904 kB 39.5 MB/s 


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

from bs4 import BeautifulSoup
import bs4
import requests
import time
from random import *
from tqdm.notebook import tqdm
import time
import pandas as pd
import numpy as np
import re
import pickle

ModuleNotFoundError: ignored

In [ ]:
# 참고 : https://cozynow.tistory.com/43
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430 Safari/537.36")

# PROXY = "socks5://127.0.0.1:9050" # tor
# chrome_options.add_argument(f'--proxy-server={PROXY}') # tor

# PROXY = "188.166.238.22:8080"
# DesiredCapabilities.CHROME['proxy'] = {
#     "httpProxy": PROXY,
#     "ftpProxy": PROXY,
#     "sslProxy": PROXY,
#     "proxyType": "MANUAL"
# }
# DesiredCapabilities.CHROME['acceptSslCerts'] = True

chrome_options.add_experimental_option("prefs", {
  # "download.default_directory": download_path,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})
driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)
# driver = webdriver.Chrome('chromedriver',options=chrome_options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472


[WDM] - Driver [/home/bjkim/.wdm/drivers/chromedriver/linux64/91.0.4472.19/chromedriver] found in cache


### 월간 차트 수집
https://www.melon.com/chart/search/index.htm

순위, 음원 제목, 가수명, 앨범명, 좋아요 수, 발매일, 장르, 댓글 수, 가사 전문, 고유번호

##### 차트 설정

In [ ]:
driver.get('https://www.melon.com/chart/search/index.htm')

In [ ]:
# 월간 차트 클릭
driver.find_element_by_css_selector('#d_chart_search > div > h4.tab02.on > a').click()

In [ ]:
# 2010년대 선택
driver.find_element_by_css_selector('#d_chart_search > div > div > div.box_chic.nth1.view > div.list_value > ul > li:nth-child(2) > span > label').click()

In [ ]:
# 연도 선택
driver.find_element_by_xpath('//label[@for="year_1"]').click()

In [ ]:
# 월 선택
driver.find_element_by_xpath('//label[@for="month_1"]').click()

In [ ]:
# 국내 종합
driver.find_element_by_xpath('//label[@for="gnr_2"]').click()

In [ ]:
# 검색 클릭
driver.find_element_by_class_name('btn_b26').click()

In [ ]:
# html 파싱
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

In [ ]:
# 음원 고유 번호
pd.Series([re.search('goSongDetail\(\'(\d+)\'\)',t['href']).group(1) for t in soup.find_all('a',{'class':'btn btn_icon_detail'})])

0     31532643
1     31532438
2     31477685
3     31406357
4     31417871
        ...   
95    31113240
96    31539246
97    31399722
98    31553909
99    31369994
Length: 100, dtype: object

In [ ]:
# 순위
[t.text for t in soup.find_all('span',{'class':'rank'})]

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '98',
 '99',
 '100']

In [ ]:
# years = reversed(range(2010,2020))
years = reversed(range(2020,2021))
months = range(1,13)

In [ ]:
df = pd.DataFrame()

In [ ]:
driver.get('https://www.melon.com/')

In [ ]:
driver.implicitly_wait(0.5)
driver.get('https://www.melon.com/chart/search/index.htm')
# 월간 차트 클릭
driver.find_element_by_css_selector('#d_chart_search > div > h4.tab02 > a').click()
# # 2020년대 선택
# driver.find_element_by_xpath('//label[@for="decade_1"]').click()
# # 2010년대 선택
driver.find_element_by_xpath('//label[@for="decade_2"]').click()

for year in tqdm(years):
    print(year)
    # 연도 선택
    driver.find_element_by_xpath(f'//label[@for="year_{2020-year}"]').click()
    for month in months:
        time.sleep(uniform(0.4,0.6))
        # 월 선택
        driver.find_element_by_xpath(f'//label[@for="month_{month}"]').click()
        # 국내 종합
        driver.find_element_by_xpath('//label[@for="gnr_2"]').click()
        # 검색 클릭
        driver.find_element_by_class_name('btn_b26').click()
        time.sleep(uniform(0.7,1.0))
        # html 파싱
        req = driver.page_source
        soup = BeautifulSoup(req, 'html.parser')
        # 순위와 음원 고유번호 파싱
        res = pd.DataFrame([re.search('goSongDetail\(\'(\d+)\'\)',t['href']).group(1) for t in soup.find_all('a',{'class':'btn btn_icon_detail'})],columns=['song_id'])
        res_rank = [t.text for t in soup.find_all('span',{'class':'rank'})]
        res.insert(0,'rank',pd.NA)
        res['rank'] = pd.Series(res_rank)
        res['rank_month'] = str(year) + str(month).zfill(2)

        df = df.append(res,ignore_index=True)

0it [00:00, ?it/s]

In [ ]:
for month in months:
    # 월 선택
    driver.find_element_by_xpath(f'//label[@for="month_{month}"]').click()
    time.sleep(uniform(0.4,0.6))
    # 국내 종합
    driver.find_element_by_xpath('//label[@for="gnr_2"]').click()
    # 검색 클릭
    driver.find_element_by_class_name('btn_b26').click()
    time.sleep(uniform(0.7,1.0))
    # html 파싱
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    # 순위와 음원 고유번호 파싱
    res = pd.DataFrame([re.search('goSongDetail\(\'(\d+)\'\)',t['href']).group(1) for t in soup.find_all('a',{'class':'btn btn_icon_detail'})],columns=['song_id'])
    res_rank = [t.text for t in soup.find_all('span',{'class':'rank'})]
    res.insert(0,'rank',pd.NA)
    res['rank'] = pd.Series(res_rank)
    res['rank_month'] = str(2020) + str(month).zfill(2)

    df = df.append(res,ignore_index=True)

In [ ]:
df

,rank,song_id,rank_month
0,1,32224272,202001
1,2,32313543,202001
2,3,32273582,202001
3,4,32183386,202001
4,5,32228058,202001
...,...,...,...
1195,96,32889003,202012
1196,97,32473998,202012
1197,98,32725022,202012
1198,99,32860419,202012


In [ ]:
df['rank_month'].value_counts()

202005    100
202002    100
202010    100
202006    100
202011    100
202004    100
202008    100
202007    100
202012    100
202003    100
202001    100
202009    100
Name: rank_month, dtype: int64

### 곡 상세 페이지 정보 수집
https://www.melon.com/song/detail.htm?songId=33515010

In [ ]:
row_list = []

In [ ]:
len(row_list)

0

In [ ]:
song_list = df['song_id'].unique()
len(song_list)

314

In [ ]:
# 상세곡 정보 수집
for song_id in tqdm(song_list):

    # 곡 페이지로 이동
    driver.get(f'https://www.melon.com/song/detail.htm?songId={song_id}')
    time.sleep(uniform(4,7))

    # html 파싱
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')

    # 노래 제목
    title = driver.find_element_by_class_name('song_name').text

    # 아티스트 고유 번호와 이름
    if soup.find('a',{'class':'artist_name'})==None:
        continue
    if re.search('goArtistDetail\(\'(\d+)\'\)',soup.find('a',{'class':'artist_name'})['href'])!=None:
        arti_id = re.search('goArtistDetail\(\'(\d+)\'\)',soup.find('a',{'class':'artist_name'})['href']).group(1)
    else:
        continue
    arti_name = soup.find('a',{'class':'artist_name'})['title']

    # 앨범 정보 (앨범 고유번호/앨범명, 발매일, 장르)
    album_id = re.search('goAlbumDetail\(\'(\d+)\'\)',soup.find_all('dd')[0].a['href']).group(1)
    album_name = soup.find_all('dd')[0].text
    date = soup.find_all('dd')[1].text
    genre = soup.find_all('dd')[2].text

    # 좋아요 수 / 댓글 수
    like = soup.find('span',{'id':'d_like_count'}).text
    comm = re.search('\d',soup.find('span',{'id':'revCnt'}).text).group()

    # 가사
    try:
        lyric = driver.find_element_by_css_selector('#d_video_summary').text
    except NoSuchElementException:
        print('가사 없음')
        continue

    # res_dict
    res = {'song_id':song_id,'title':title, 'arti_id':arti_id, 'arti_name':arti_name, \
        'album_id':album_id, 'album_name':album_name, 'date':date, 'genre':genre,\
            'like':like, 'comm':comm, 'lyric':lyric}

    # merge row
    row_list.append(res)

  0%|          | 0/314 [00:00<?, ?it/s]

가사 없음


In [ ]:
row_list

라 반드시\n버텨 보란 듯이\n방안에 갇힌 누나까지\n동네의 친구 선생 부장까지\n유년시절 기억까지\n아버지 아버지\n어머니 어머니\n내가 입던 교복은 다 타버리고 있지\nEvery time I ride my plan\n빠르지 네 차보다 더\n배춧잎 속에 살던 넌\n속았어 속았어\nEvery time we were looking for you\n드디어 찾았어 Ok\nTricker tricker tricker tricker\n부어라 비워라\n부어라 비워라\n부어라 비워라\n부어라 비워라\n부어라 비워라\n부어라 비워라\nDo you wanna be the one\n요동쳐 투쟁의 리듬\n미래는 광복의 또 다른 이름\n생기를 잃은\n이 거리 위에서\nHigh way\nFreeway\nLiberate\n남겨라 가죽과 이름\n지켜라 너의 가족 모든 이를\n광합성을 놓친\n이 도시 위에서\nHigh way\nFreeway\nLiberate\nFreedom\nLet’s Go\nOk tricker checkin’\n내 눈알의 색\n만들어 Track\nYes I’m always\n평생을 바쳤던 만큼\n울려라 Bass\n울려라 승전보 Ha\n단물 빠지면은 토사구팽\n세상에 Say\n세상에 Say\nI don’t care 난 너네 우월감 위에서\n네 선입견 깨버릴 준비해\n자유를 쥔채 Take is it\n말대꾸 Take it back\n투쟁은 또 다른 분쟁을\n새로운 주제는 만들어 문제를 그렇담\nBrr ta ta tah\n절심함 속에서 여기까지 혼자\nI’m on the new level\n약이나 팔아라 난 악으로 살아\n나의 적들에게 Trauma\n난 개무시 널 개무시\n찢어발겨 버려 너네 부심\n작지만 커지는 투쟁의 힘\n생성시켜 아이들 이끌 힘을\n머쉬베놈에게 구원자 같은\n역할 했던 것은 맞아 음악\n그런데 뭐하나 보여준 거 하나\n없이 떠들어봐\n거리에 배째고 누워 난\nLet’s get it'},
 {'song_id': '33133799',
  'title': '고

In [ ]:
df_song = pd.DataFrame(row_list)
df_song

,song_id,title,arti_id,arti_name,album_id,album_name,date,genre,like,comm,lyric
0,32224272,METEOR,774900,창모 (CHANGMO),10359162,Boyhood,2019.11.29,랩/힙합,"261,343",5,Yeah ha\nYeah ha\nYeah ha\nYeah ha\nYeah ha\nY...
1,32313543,아무노래,602056,지코 (ZICO),10375118,아무노래,2020.01.13,랩/힙합,"256,285",3,왜들 그리 다운돼있어\n뭐가 문제야 say something\n분위기가 겁나 싸해\...
2,32273582,Psycho,780066,Red Velvet (레드벨벳),10368053,‘The ReVe Festival’ Finale,2019.12.23,댄스,"213,754",6,Psycho\n\n널 어쩌면 좋을까\n이런 맘은 또 첨이라\nUp & Down이 좀...
3,32183386,Blueming,261143,아이유,10346650,Love poem,2019.11.18,록/메탈,"307,057",2,‘뭐해?‘라는 두 글자에\n‘네가 보고 싶어’ 나의 속마음을 담아 우\n이모티콘 하...
4,32228058,"아마두 (feat.우원재, 김효은, 넉살, Huckleberry P)",202530,염따,10359994,Dingo X DAMOIM (Part 2),2019.12.03,랩/힙합,"155,752",3,어 그래 얘들아\n사는게 개같지\n내 얼굴도 개같지 뭐얌\n근데도 부자가 됐다\n세...
...,...,...,...,...,...,...,...,...,...,...,...
308,33077238,잠시,672375,방탄소년단,10521521,BE,2020.11.20,랩/힙합,"101,807",1,매번 같은 하루들 중에\n너를 만날 때 가장 난 행복해\n매번 다른 일상들 속에\n...
309,33129712,잘할게,58793,이승기,10532045,The Project,2020.12.10,발라드,"28,417",2,우우우우 우우우우우\n우우우우 그게 잘 안돼 그게 잘 안돼\n우우우우 우우우우우\n...
310,33077240,Stay,672375,방탄소년단,10521521,BE,2020.11.20,댄스,"92,229",1,꿈이었을까\n널 본 것 같아\n눈뜨면 다시\n아무도 없는 방\n\n가만히 난 주문을...
311,33077239,병,672375,방탄소년단,10521521,BE,2020.11.20,랩/힙합,"93,037",1,뭔가 놓친듯해\n커피 한 모금으로 불안함을 해소\nAn endless rest\n내...


In [ ]:
df_song['like'] = df_song['like'].str.replace(',','').astype(int)
df_song['comm'] = df_song['comm'].str.replace(',','').astype(int)

AttributeError: Can only use .str accessor with string values!

In [ ]:
df_song['date'] = pd.to_datetime(df_song['date'])

In [ ]:
# merge
df_merged = df.merge(df_song,how='right',on='song_id')
df_merged

,rank,song_id,rank_month,title,arti_id,arti_name,album_id,album_name,date,genre,like,comm,lyric
0,1,32224272,202001,METEOR,774900,창모 (CHANGMO),10359162,Boyhood,2019-11-29,랩/힙합,261343,5,Yeah ha\nYeah ha\nYeah ha\nYeah ha\nYeah ha\nY...
1,2,32224272,202002,METEOR,774900,창모 (CHANGMO),10359162,Boyhood,2019-11-29,랩/힙합,261343,5,Yeah ha\nYeah ha\nYeah ha\nYeah ha\nYeah ha\nY...
2,5,32224272,202003,METEOR,774900,창모 (CHANGMO),10359162,Boyhood,2019-11-29,랩/힙합,261343,5,Yeah ha\nYeah ha\nYeah ha\nYeah ha\nYeah ha\nY...
3,9,32224272,202004,METEOR,774900,창모 (CHANGMO),10359162,Boyhood,2019-11-29,랩/힙합,261343,5,Yeah ha\nYeah ha\nYeah ha\nYeah ha\nYeah ha\nY...
4,13,32224272,202005,METEOR,774900,창모 (CHANGMO),10359162,Boyhood,2019-11-29,랩/힙합,261343,5,Yeah ha\nYeah ha\nYeah ha\nYeah ha\nYeah ha\nY...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,83,33077238,202012,잠시,672375,방탄소년단,10521521,BE,2020-11-20,랩/힙합,101807,1,매번 같은 하루들 중에\n너를 만날 때 가장 난 행복해\n매번 다른 일상들 속에\n...
1192,86,33129712,202012,잘할게,58793,이승기,10532045,The Project,2020-12-10,발라드,28417,2,우우우우 우우우우우\n우우우우 그게 잘 안돼 그게 잘 안돼\n우우우우 우우우우우\n...
1193,92,33077240,202012,Stay,672375,방탄소년단,10521521,BE,2020-11-20,댄스,92229,1,꿈이었을까\n널 본 것 같아\n눈뜨면 다시\n아무도 없는 방\n\n가만히 난 주문을...
1194,93,33077239,202012,병,672375,방탄소년단,10521521,BE,2020-11-20,랩/힙합,93037,1,뭔가 놓친듯해\n커피 한 모금으로 불안함을 해소\nAn endless rest\n내...


In [ ]:
# df.to_pickle('2010s_melon_rank.pkl')
# df_song.to_pickle('2010s_melon_rank_song.pkl')
# df_merged.to_pickle('2010s_melon_rank_song_merged.pkl')

df.to_pickle('2020_melon_rank.pkl')
df_song.to_pickle('2020_melon_rank_song.pkl')
df_merged.to_pickle('2020_melon_rank_song_merged.pkl')